# Simple Explicit Baseline
* Computes a bias for each user and for each item
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = m + u_i + a_j$
* $m = \text{mean}_{ij}(r_{ij})$
* $u_i = \text{mean}_j(r_{ij}) - m$
* $a_j = \text{mean}_i(r_{ij}) - m$
* $r_{ij}$ is the rating for user $i$ and item $j$
* Useful as a benchmark to compare more sophicated algorithms against

In [ ]:
medium = ""
task = "";

In [ ]:
const name = "$medium/$task/SimpleExplicitBaseline"
const residual_alphas = String[]
const content = "explicit"
const implicit = false;

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");

In [ ]:
using DataFrames
import Statistics: mean

## Training

In [ ]:
function get_residualized_dataframe(split, task, content, residual_alphas)
    df = get_split(split, task, content, medium)
    residuals = read_alpha(residual_alphas, split, task, content, medium, implicit)
    DataFrame(user = df.user, item = df.item, rating = df.rating - residuals.rating)
end

const training = get_residualized_dataframe("training", task, content, residual_alphas);

In [ ]:
const μ = mean(training.rating);

In [ ]:
u = combine(groupby(training, :user), :rating => mean => :rating)
u = Dict(Pair.(u.user, u.rating .- μ));

In [ ]:
a = combine(groupby(training, :item), :rating => mean => :rating)
a = Dict(Pair.(a.item, a.rating .- μ));

## Inference

In [ ]:
function make_prediction(users, items, u, a, μ)
    r = Array{eltype(μ)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        rating = μ
        if users[i] in keys(u)
            rating += u[users[i]] 
        end
        if items[i] in keys(a)
            rating += a[items[i]]
        end        
        r[i] = rating
    end
    r
end;

model(users, items) = make_prediction(users, items, u, a, μ);

In [ ]:
write_alpha(
    model,
    medium,
    name;
    log = true,
    log_task = task,
    log_content = content,
    log_alphas = residual_alphas,
)

In [ ]:
write_params(Dict("u" => u, "a" => a, "μ" => μ), name);